In [1]:
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessor,LogitsProcessorList

import torch
from transformers import BitsAndBytesConfig
from tqdm import tqdm



/work/pi_mccallum_umass_edu/aparashar_umass_edu/.conda/envs/as01/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
qa_exemplars = [
    {
        "question": "Do hamsters provide food for any animals?",
        "reasoning": "Hamsters are prey animals. Prey are food for predators. Thus, hamsters provide food for some animals. So the answer is yes.",
        "answer" : "Yes"
    },
    {
        "question": "Could Brooke Shields succeed at University of Pennsylvania?",
        "reasoning": "Brooke Shields went to Princeton University. Princeton University is about as academically rigorous as the University of Pennsylvania. Thus, Brooke Shields could also succeed at the University of Pennsylvania. So the answer is yes.",
        "answer" : "Yes"
    },
    {
        "question": "Hydrogen’s atomic number squared exceeds number of Spice Girls?",
        "reasoning": "Hydroßen has an atomic number of 1. 1 squared is 1. There are 5 Spice Girls. Thus, Hydrogen’s atomic number squared is less than 5. So the answer is no.",
        "answer" : "No"
    },
    {
        "question": "Is it common to see frost during some college commencements?",
        "reasoning": "College commencement ceremonies can happen in December, May, and June. December is in the winter, so there can be frost. Thus, there could be frost at some commencements. So the answer is yes.",
        "answer" : "Yes"
    },
    {
        "question": "Could a llama birth twice during War in Vietnam (1945-46)?",
        "reasoning": "The War in Vietnam was 6 months. The gestation period for a llama is 11 months, which is more than 6 months. Thus, a llama could not give birth twice during the War in Vietnam. So the answer is no.",
        "answer" : "No"
    },
    {
        "question": "Would a pear sink in water?",
        "reasoning": "The density of a pear is about 0.6 g/cm3, which is less than water. Objects less dense than water float. Thus, a pear would float. So the answer is no.",
        "answer" : "No"
    }
]


In [3]:
# def run_experiments():
#     tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b", use_fast=True)
#     model = AutoModelForCausalLM.from_pretrained("google/gemma-2").to('cuda')
#     torch.device = 'cuda'
#     # set pad_token_id to eos_token_id because GPT2 does not have a EOS token
#     model.config.pad_token_id = model.config.eos_token_id
#     model.generation_config.pad_token_id = model.config.eos_token_id
#     data = random.sample(load_hf_data_set('validation','wmt14','de-en')['translation'],200)
    
#     default_fwd_instructi
#     on = f"Yes or No: {question}"
#     default_fwd_input_prefix = "Yes or No: "
#     default_fwd_target_prefix = "Answer: "
#     demos = random.sample(qa_pairs,5)
#     demos = [[default_fwd_input_prefix + d["question"]] +'\n'  [default_fwd_target_prefix + d["answer"]] for d in demos]
#     prompt_arr = demos.append([default_fwd_instruction,default_fwd_input_prefix])
#     for N in [5, 10]:
#         for temp in [0.5, 1.0]:
#             test(prompt_arr, model, tokenizer, data, default_fwd_target_prefix, N, temp)


In [28]:
from typing import Union,List

class TruncateLogitsProcessor(LogitsProcessor):
    def __init__(self,token_id: Union[int, List[int]],stop_word:Union[str, List[str]],eos_token_id: Union[int, List[int]],tokenizer):
        if isinstance(token_id, int):
            token_id = [token_id]
        if isinstance(eos_token_id, int):
            eos_token_id = [eos_token_id]
        # if not all(isinstance(i,int) for i in token_id) or any(i < 0 for i in token_id):
        #     logger.warning(f"`token_id` has to be a list of positive integers, but is {token_id}")
        self.stop_word = stop_word
        self.token_id = token_id
        
        self.eos_token_id = eos_token_id
        self.tokenizer = tokenizer
    def __call__(self,input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        cur_len = input_ids.shape[-1]
        
        # print(cur_len)
        # print(scores.shape)
        max_score_ids = torch.argmax(scores, dim=1)
        
        for i in range(len(max_score_ids)):
            #print(tokenizer.decode(max_score_ids[i]))
            if (max_score_ids[i] in tokenizer(self.stop_word).input_ids or self.tokenizer.decode(max_score_ids[i]).find(self.stop_word) != -1) :
                print("GITTT")
                scores[i][:] = -float('inf')
                scores[i][self.eos_token_id[0]] = 0
        #print(torch.argmax(scores, dim=1))
        # print(input_ids.shape)
        #print(input_ids)
        scores.to(input_ids.device)
        if torch.argmax(scores[:,]) in [894]:
            print('yes')
            scores = torch.zeros(scores.shape)
            scores[:, self.eos_token_id] = 1
                
        return scores



In [29]:
tokenizer.decode([894,45])

'Question*'

In [ ]:
# default_instruction = "Answer the question with Yes or No: "
default_fwd_input_prefix = """Like each of the previous examples, answer the question with either "Yes" or "No".
Provide reasoning for your answer.
End your reasoning with the sentence: "So the answer is <your_answer>". Replace "<your_answer>" with your final answer, which should be either "Yes" or "No"."""

default_fwd_demos_prefix = "Here are some examples of reasoning based question answers."
default_fwd_reasoning_prefix = "Reasoning: "
default_fwd_question_prefix = "Question: "
default_fwd_target_prefix = "Answer: "
demos = random.sample(qa_exemplars,6)
demos = [[default_fwd_question_prefix + d["question"]]+ [default_fwd_reasoning_prefix + d["reasoning"] + default_fwd_target_prefix + d["answer"]] for d in demos]# def test(prompt_arr, model, tokenizer, data,demos, default_fwd_target_prefix, N = 10, temp = 0.5):
bnb_config= BitsAndBytesConfig(
            load_in_8bit=True,)
output_dict = {}
import json
filepath = "/work/pi_dhruveshpate_umass_edu/aparashar_umass_edu/Arithmetic-Sampling/data/stratqa_data.json"
with open(filepath,'r') as f:
    strat_data = json.load(f)['examples']
    
demo_questions = [d['question'] for d in qa_exemplars]
print(len(strat_data))
strat_data = [i for i in strat_data if i['input'] not in demo_questions]
print(len(strat_data))
# device_map="auto"
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b", use_fast=True)
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b",
                                             # torch_dtype=torch.bfloat16,
                                             cache_dir = '/work/pi_mccallum_umass_edu/aparashar_umass_edu/models/.cache',
                                             # device_map="auto",
                                                # quantization_config=bnb_config
                                            ).to('cuda')
logits_processor = LogitsProcessorList(
    [
        #StopAfterTokenLogitsProcessor(token_id_to_stop=29973,max_length=40),
        TruncateLogitsProcessor(stop_word='Question',token_id=[tokenizer.encode('Question')],eos_token_id=tokenizer.eos_token_id,tokenizer=tokenizer),
    ])
# torch.device = 'cuda'
# set pad_token_id to eos_token_id because GPT2 does not have a EOS token
model.config.pad_token_id = model.config.eos_token_id
model.generation_config.pad_token_id = model.config.eos_token_id
for idx, d in enumerate(tqdm(strat_data[1:7], desc="Predicting")):
    print(d)
    prompt_arr = [default_fwd_input_prefix] +[default_fwd_question_prefix + d['input']]+[default_fwd_reasoning_prefix]
    prompt_arr = demos + [prompt_arr]
    prompt_arr = [element for sublist in prompt_arr for element in sublist]
    # print(prompt_arr)
    input_prompt =  ('\n').join(prompt_arr)
    print(f"prompt: {input_prompt}")
    input_ids = tokenizer(input_prompt, truncation=True, return_tensors="pt").input_ids.to('cuda')
    outputs_arith = model.generate(
        input_ids = input_ids,
        num_return_sequences = 5,
        do_sample = True,
        max_new_tokens = 200,
        temperature = 1.,
        num_beams = 1,
        return_dict_in_generate=True,
        logits_processor=logits_processor,
        use_arithmetic = True
        )
    outputs_sample = model.generate(
        input_ids = input_ids,
        num_return_sequences = 5,
        do_sample = True,
        temperature = 1.,
        num_beams = 1,
        max_new_tokens = 200,
        return_dict_in_generate=True,
        logits_processor=logits_processor,
        
        use_arithmetic = False
        )
    print(input_ids.shape[1])
    outputs_arith = [tokenizer.decode(o, skip_special_tokens=True)[len(input_prompt):] for o in outputs_arith.sequences]
    outputs_sample = [tokenizer.decode(o, skip_special_tokens=True)[len(input_prompt):] for o in outputs_sample.sequences]
    # print(f"Arithmetic output: {tokenizer.batch_decode(outputs_arith[input_ids.shape[1]:], skip_special_tokens=True)}")
    outputs_arith_str = '\n###\n'.join(outputs_arith)
    print("---")
    outputs_sample_str = '\n###\n'.join(outputs_sample)
    print(f"Arithmetic output: {outputs_arith_str}")
    print(f"Sample output: {outputs_sample_str}")
    

2290
2285


Predicting:   0%|                                                                                                                                                            | 0/6 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input': 'Are all limbs required for jujutsu?', 'target_scores': {'Yes': 0, 'No': 1}, 'target': 'No. Jujutsu is a Japanese martial art that uses unarmed combat to subdue opponents. Nick Newell, a congenital amputee, got his black belt after two straight submission wins. Fighter Aaron LaPointe has succeeded in martial arts with a fully paralyzed arm.'}
prompt: Question: Is it common to see frost during some college commencements?
Reasoning: College commencement ceremonies can happen in December, May, and June. December is in the winter, so there can be frost. Thus, there could be frost at some commencements. So the answer is yes.Answer: Yes
Question: Could Brooke Shields succeed at University of Pennsylvania?
Reasoning: Brooke Shields went to Princeton University. Princeton University is about as academically rigorous as the University of Pennsylvania. Thus, Brooke Shields could also succeed at the University of Pennsylvania. So the answer is yes.Answer: Yes
Question: Would a pear sink

Predicting:  17%|████████████████████████▋                                                                                                                           | 1/6 [00:32<02:41, 32.31s/it]

GITTT
454
---
Arithmetic output: 5 limbs is about the same number of limbs as jujutsu. Thus, there are probably jujutsu techniques that require 5 limbs. So the answer is yes.Answer: Yes

###
1. Jujutsu does not define the term "limb" 2. Budo arts do not require specific limb parts or locations. Thus, they are not required for jujutsu. 3. Some martial arts require a specific arm, leg (i.e., wrist, elbow, knee, ankle or other) or body part such as the face, eyes, mouth, or heart to perform kicks, punches, jiu-jitsu techniques or attacks. Thus, they are not required for budo arts. 4. Many martial arts use all or some of the arms, legs (i.e., wrist, elbow, knee, ankle or other) and body parts. Thus, they are not restricted or limited to just one limb part or location. Therefore, in my opinion, all parts or locations of the limbs are not required for jujutsu. The answer is "No".Answer: No

###
17 limbs (4 arms, 4 legs, and 9 organs) can be used to make any shape. That includes making differ

...

GITTT
457
---
Arithmetic output: <b>Linus Torvalds</b> founded Linux and Microsoft's DirectX licenses from his company and is well known for not making a dime off them. Thus, <b>Linus Torvalds </b>does not make money off of DirectX. Answer: No

###
1. Linus Torvalds is the developer of Linux, which is very popular. 2. Linus's parents are rich. The fact that his parents are rich makes Linus Torvalds become rich. We all know that rich people get rich money. 3. Thus, Linus Torvalds makes money off of his code that makes Linux so popular. So, the answer is yes. Answer: Yes

###
17 years ago Linus Torvalds, creator of the popular Linux operating system, created the kernel in response to a need. This led to the creation of Microsoft’s DirectX graphic package. Thus, people could theoretically make money off of the popularity of DirectX. So the answer is yes.Answer: Yes

###
28 years ago, Linus Torvalds started working on the Linux operating system. The price of Linux is much cheaper than the 

Predicting:  50%|██████████████████████████████████████████████████████████████████████████                                                                          | 3/6 [01:25<01:26, 28.83s/it]

GITTT
459
---
Arithmetic output: 1. The Empire State Building is 1,454 feet tall. 1.454 cubed is 108,196.477. 108,196.477 pounds is more than 300 kg. 300 kg is about 660 pounds. Thus, a silverfish could definitely reach the top of the Empire State Building. So the answer is yes.
Question: Is this true: "An elephant has more hairs per pound than a rabbit has hairs per kilogram"?
Reasoning: 1. Elephants live around 50 years. 1.43 times 50 is 73.7. 73.7 to the power 4 is 3.7647. 3.7647 pounds is 18 kilograms. 5 to the power -5 is 1.8. Thus, no. So the
###
1000m x 200m x 1m = 0.2 km^3 (0.2 cubic kilometer). The largest volume that the largest silverfish can occupy is 0.2 cubic kilometers. This assumes that the silverfish is completely compacted, which is very unlikely. If the silverfish had 1 meter from all sides, then it would have a volume of 0.512 cubic centimeters (100cm x 100cm x 100cm). Thus, the density of the silverfish is 5.49g/cm^3. 0.512 cubic centimeters = .512 x 5.49 = 2.81 g.

Predicting:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████▋                                                 | 4/6 [01:45<00:50, 25.48s/it]

GITTT
460
---
Arithmetic output: 5 million Reddit users post something daily. Many of those posts are from people that do not live in the United States. This means that many of the people using Reddit are not familiar with the Pledge of Allegiance. Thus, the majority of Reddit users are not familiar with the Pledge of Allegiance.So the answer is no.Answer: No

###
81.2 percent of the subreddit Reditt is a part of Reddit.com is based in the United States. The national pledge of allegiance says "I pledge allegiance." All 50 United States have the same pledge of allegiance. Thus, the majority of Reddit users are familiar with the pledge of allegiance. So the answer is yes.Answer: Yes

###
17.9 million Reddit users participated in the first Reddit Poll in 2005. That poll lasted 3 hours. Assume that approximately the same number of participants voted in 95% of all Reddit Polls. Thus, 16.6 million Reddit user answered the question "Is the majority of Reddit users familiar with the Pledge of 

In [ ]:
import json
filepath = "/work/pi_dhruveshpate_umass_edu/aparashar_umass_edu/Arithmetic-Sampling/data/stratqa_data.json"
with open(filepath,'r') as f:
    strat_data = json.load(f)['examples']
demo_questions = [d['question'] for d in demos]
print(len(strat_data))
strat_data = [i for i in strat_data if i['input'] not in demo_questions]
print(len(strat_data))

In [7]:
strat_data[0]

{'input': "Hydrogen's atomic number squared exceeds number of Spice Girls?",
 'target_scores': {'Yes': 0, 'No': 1},
 'target': 'No. Hydrogen is the first element and has an atomic number of one. To square a number, you multiply it by itself. The Spice Girls has five members.'}

In [5]:
pwd

'/work/pi_dhruveshpate_umass_edu/aparashar_umass_edu/Arithmetic-Sampling/notebooks'

In [13]:
strat_data[0]

{'input': 'Is it common to see frost during some college commencements?',
 'target_scores': {'Yes': 1, 'No': 0},
 'target': "Yes. College commencement ceremonies often happen during the months of December, May, and sometimes June.  Frost isn't uncommon to see during the month of December, as it is the winter."}

In [19]:
!pip show bitsandbytes  


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: bitsandbytes
Version: 0.43.0
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /work/pi_mccallum_umass_edu/aparashar_umass_edu/.conda/envs/as01/lib/python3.10/site-packages
Requires: numpy, torch
Required-by: 
